In [1]:
import os
import zipfile
# import random
# import shutil
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from shutil import copyfile
# import matplotlib.pyplot as plt
# import psutil
# import time

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [2]:
train_data_path = "train"
validation_data_path = "valid"
test_data_path = "test"

In [3]:
img_width, img_height = 150, 150
# channel = (3, )
input_shape = (img_width, img_height)
# input_shape = tf.expand_dims(input_shape , -1)
batch_size = 64

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'

)

validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'

)

test_datagen = ImageDataGenerator(rescale=1.0 / 255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  fill_mode='nearest'

                                  )

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(test_data_path,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 260915 images belonging to 14 classes.
Found 47531 images belonging to 14 classes.
Found 137375 images belonging to 14 classes.


In [6]:
def tf_data_generator(generator, input_shape):
  num_class = generator.num_classes
  print(num_class)

  # Define a separate function for generating data
  def _generator():
    for images, labels in generator:
      yield images, labels

  # Create the tf.data.Dataset from the non-decorated function
  tf_generator = tf.data.Dataset.from_generator(
      _generator,
      output_types=(tf.float32, tf.float32),
      output_shapes=([None, input_shape[0], input_shape[1], None],
                     [None, num_class])
  )

  return tf_generator


train_data_generator = tf_data_generator(train_generator, input_shape)
test_data_generator = tf_data_generator(test_generator, input_shape)
val_data_generator = tf_data_generator(validation_generator, input_shape)

14
14
14


In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Conv2D, Activation

In [8]:
def create_model(img_width, img_height, num_classes):

    model = Sequential()
    model.add(Conv2D(128, (3, 3), padding='same',
              input_shape=(img_width, img_height, 3)))

    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(14))
    model.add(Activation('softmax'))

    return model


model = create_model(img_width, img_height, 14)

In [9]:
# def create_model(img_width, img_height, num_classes):

#     base_model = MobileNetV2(
#         input_shape=input_shape + (3,),
#         include_top=False,
#         weights='imagenet',
#         classes=num_classes,
#     )

#     x = base_model.output
#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dropout(0.2)(x)

#     x = layers.Dense(1024, activation="relu")(x)

#     predictions = layers.Dense(num_classes, activation="softmax")(x)
#     model = Model(inputs=base_model.input, outputs=predictions)

#     return model


# model = create_model(img_width, img_height, 14)

In [10]:
# opt = SGD(lr=0.01, momentum=0.9)
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    # optimizer=opt,
    metrics=["accuracy", "categorical_crossentropy"],
)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss", patience=3)

history1 = model.fit(
    train_data_generator,
    steps_per_epoch=len(train_generator) // train_generator.batch_size,
    epochs=30,
    validation_data=val_data_generator,
    validation_steps=len(
        validation_generator) // validation_generator.batch_size,
    callbacks=[early_stopping],
)

Epoch 1/30


27/63 [===========>..................] - ETA: 2:56 - loss: 2.6966 - accuracy: 0.1372 - categorical_crossentropy: 2.6966

KeyboardInterrupt: 